In [1]:
from misc import HP
import argparse
import random
import time
import pickle
import copy
import SYCLOP_env as syc
from misc import *
import sys
import os
import cv2
import argparse
import tensorflow.keras as keras

from keras_networks import rnn_model_102e
from curriculum_utils import create_mnist_dataset, bad_res102

# ArgumentParser()
def split_dataset_xy(dataset):
    dataset_x1 = [uu[0] for uu in dataset]
    dataset_x2 = [uu[1] for uu in dataset]
    dataset_y = [uu[-1] for uu in dataset]
    return (np.array(dataset_x1)[...,np.newaxis],np.array(dataset_x2)[:,:n_timesteps,:]), np.repeat(np.reshape(dataset_y,[-1,1]),hp.steps_per_episode,axis=1)

#parse hyperparameters

lsbjob = os.getenv('LSB_JOBID')
lsbjob = '' if lsbjob is None else lsbjob

hp = HP()
hp.save_path = 'saved_runs'
hp.description=''
parser = argparse.ArgumentParser()
parser.add_argument('--tau_int', default=4., type=float, help='Integration timescale for adaaptation')
parser.add_argument('--resize', default=1.0, type=float, help='resize of images')
parser.add_argument('--run_name_suffix', default='', type=str, help='suffix for runname')
parser.add_argument('--eval_dir', default=None, type=str, help='eval dir')

parser.add_argument('--dqn_initial_network', default=None, type=str, help='dqn_initial_network')
parser.add_argument('--decoder_initial_network', default=None, type=str, help='decoder_initial_network')
parser.add_argument('--decoder_learning_rate',  default=1e-3, type=float, help='decoder learning rate')
parser.add_argument('--decoder_dropout',  default=0.0, type=float, help='decoder dropout')
parser.add_argument('--decoder_rnn_type',  default='gru', type=str, help='gru or rnn')
parser.add_argument('--decoder_rnn_units',  default=100, type=int, help='decoder rnn units')
parser.add_argument('--decoder_rnn_layers',  default=1, type=int, help='decoder rnn units')


parser.add_argument('--decoder_ignore_position', dest='decoder_ignore_position', action='store_true')
parser.add_argument('--no-decoder_ignore_position', dest='decoder_ignore_position', action='store_false')

parser.add_argument('--syclop_learning_rate',  default=2.5e-3, type=float, help='syclop (RL) learning rate')

parser.add_argument('--color', default='grayscale', type=str, help='grayscale/rgb')
parser.add_argument('--speed_reward',  default=0.0, type=float, help='speed reward, typically negative')
parser.add_argument('--intensity_reward',  default=0.0, type=float, help='speed penalty reward')
parser.add_argument('--loss_reward',  default=-1.0, type=float, help='reward for loss, typically negative')
parser.add_argument('--resolution',  default=6, type=int, help='resolution')
parser.add_argument('--max_eval_episodes',  default=10000, type=int, help='episodes for evaluation mode')
parser.add_argument('--steps_per_episode',  default=5, type=int, help='time steps in each episode in ')
parser.add_argument('--fit_verbose',  default=1, type=int, help='verbose level for model.fit                        ')
parser.add_argument('--steps_between_learnings',  default=100, type=int, help='steps_between_learnings')
parser.add_argument('--num_epochs',  default=100, type=int, help='steps_between_learnings')

parser.add_argument('--alpha_increment',  default=0.01, type=float, help='reward for loss, typically negative')


parser.add_argument('--beta_t1',  default=400000, type=int, help='time rising bete')
parser.add_argument('--beta_t2',  default=700000, type=int, help='end rising beta')
parser.add_argument('--beta_b1',  default=0.1, type=float, help='beta initial value')
parser.add_argument('--beta_b2',  default=1.0, type=float, help='beta final value')

parser.add_argument('--curriculum_enable', dest='curriculum_enable', action='store_true')
parser.add_argument('--no-curriculum_enable', dest='curriculum_enable', action='store_false')

parser.add_argument('--conv_fe', dest='conv_fe', action='store_true')
parser.add_argument('--no-conv_fe', dest='conv_fe', action='store_false')

parser.add_argument('--acceleration_mode', dest='acceleration_mode', action='store_true')
parser.add_argument('--no-acceleration_mode', dest='acceleration_mode', action='store_false')


parser.set_defaults(eval_mode=False, decode_from_dvs=False,test_mode=False,rising_beta_schedule=True,decoder_ignore_position=False, curriculum_enable=True, conv_fe=False,
                    acceleration_mode=False)

config = parser.parse_args('')
config = vars(config)
hp.upadte_from_dict(config)
hp.this_run_name = sys.argv[0] + '_noname_' + hp.run_name_suffix + '_' + lsbjob + '_' + str(int(time.time()))

#define model
n_timesteps = hp.steps_per_episode

##
# deploy_logs()
##
decoder = rnn_model_102e(lr=hp.decoder_learning_rate,ignore_input_B=hp.decoder_ignore_position,dropout=hp.decoder_dropout,rnn_type=hp.decoder_rnn_type,
                                input_size=(hp.resolution,hp.resolution, 1),rnn_layers=hp.decoder_rnn_layers,conv_fe=hp.conv_fe)
# decoder = keras.models.load_model(hp.decoder_initial_network)
#define dataset
(images, labels), (images_test, labels_test) = keras.datasets.mnist.load_data(path="mnist.npz")


#fit one epoch in a  time
# scheduler = Scheduler(hp.lambda_schedule)
# for epoch in range(hp.num_epochs):
#     lambda_epoch = scheduler.step(epoch)
alpha=0
for epoch in range(hp.num_epochs):
    if hp.curriculum_enable:
        if epoch == 0:
            train_dataset, test_dataset = create_mnist_dataset(images, labels, 6, bad_res_func=bad_res102, return_datasets=True, q_0=0, alpha=1.0,random_trajectories=False,acceleration_mode=hp.acceleration_mode)
            train_dataset_x, train_dataset_y = split_dataset_xy(train_dataset)
            test_dataset_x, test_dataset_y = split_dataset_xy(test_dataset)
            q_0=train_dataset_x[1][0]
            print('debu0',q_0)
        else:
            alpha += hp.alpha_increment
            alpha = np.minimum(alpha,1.0)

            train_dataset, test_dataset = create_mnist_dataset(images, labels, 6, bad_res_func=bad_res102, return_datasets=True, q_0=q_0, alpha=alpha,random_trajectories=True,acceleration_mode=hp.acceleration_mode)
            train_dataset_x, train_dataset_y = split_dataset_xy(train_dataset)
            test_dataset_x, test_dataset_y = split_dataset_xy(test_dataset)
            q_prime=train_dataset_x[1][0]
            print('epoch',epoch,' alpha',alpha,'first q --', q_prime.reshape([-1]))
    else:
        train_dataset, test_dataset = create_mnist_dataset(images, labels, 6, bad_res_func=bad_res102,
                                                           return_datasets=True, q_0=0, alpha=1.0,
                                                           random_trajectories=True,acceleration_mode=hp.acceleration_mode)
        train_dataset_x, train_dataset_y = split_dataset_xy(train_dataset)
        test_dataset_x, test_dataset_y = split_dataset_xy(test_dataset)
        q_prime = train_dataset_x[1][0]
        print('epoch', epoch, '  CONTROL!!!',' first q --', q_prime.reshape([-1]))
        print('epoch', epoch, '  CONTROL!!!',' first im --', train_dataset_x[0][0])

    print("Fit model on training data")
    history = decoder.fit(
        train_dataset_x,
        train_dataset_y,
        batch_size=135,
        epochs=1,
        verbose=hp.fit_verbose,
        # We pass some validation for
        # monitoring validation loss and metrics
        # at the end of each epoch
        validation_data=(test_dataset_x, test_dataset_y)  # (validation_images, validation_labels)
        )

#save the model



(?, 5, 38)
debu return sequence True
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
debug_shapes: (?, 5, 10)
debug_shapes: (?, 5)
debug_shapes: (?, 5, 10)
debug_shapes: (?, 5)

Are we random? 18
debu0 [[36 36]
 [37 41]
 [37 41]
 [37 46]
 [32 46]]
Fit model on training data
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 55000 samples, validate on 5000 samples
55000/55000 [==============================] - 7s 126us/sample - loss: 2.0993 - time_distributed_accuracy: 0.2507 - time_distributed_accuracy_last_step: 0.2599 - val_loss: 1.7162 - val_time_distributed_accuracy: 0.4121 - val_time_distributed_accuracy_last_step: 0.4774
Are we random? 12
epoch 1  alpha 0.01 first q -- [36 36 37 40 37 40 37 45 32 45]
Fit model on training data
Train on 55000 samples, validate on 5000 samples
55000/55000 [==============================] - 6s 116us/sample - loss: 1.6782 - time_distributed_accuracy: 0.4203 - time_di

KeyboardInterrupt: 

In [ ]:
argparse.__version__

In [ ]:
config = parser.parse_args('')


In [ ]:
train_dataset_y.shape

In [ ]:
            train_dataset, test_dataset = create_mnist_dataset(images, labels, 6, bad_res_func=bad_res102, return_datasets=True, q_0=0, alpha=1.0,random_trajectories=False,acceleration_mode=hp.acceleration_mode)
            train_dataset_x, train_dataset_y = split_dataset_xy(train_dataset)
            test_dataset_x, test_dataset_y = split_dataset_xy(test_dataset)